In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#started again because i suspected there was a problem in my data with all the overcomplicated bs i was doing with my numpy arrays

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
#read data
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

In [ ]:
train.head()

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#create df for malignant and benign samples
mal_train = train[train['target']==1]
ben_train = train[train['target']==0].sample(n=2000, random_state = 316)

In [ ]:
#combine and change image_name to directory values
total_train =  pd.concat([mal_train, ben_train], ignore_index=True, sort =False)
total_train['image_name'] = train_dir + total_train['image_name'] + '.jpg'

#remove unnecessary columns
total_train = total_train.drop(['patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge', 'diagnosis', 'benign_malignant'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#split into validation and train
x_train, x_val, y_train, y_val = train_test_split(total_train['image_name'], total_train['target'], test_size = 0.2, shuffle = True, random_state = 316)

In [ ]:
#creating dataframes to feed into ImageDataGenerator with flow_from_dataframe
train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})

In [ ]:
#convert all the directories to strings
train_gen['image_dir'].astype(str)
val_gen['image_dir'].apply(str)

In [ ]:
from albumentations import (Compose,
                            IAAAdditiveGaussianNoise, IAAAffine, IAAPerspective, IAASharpen, 
                            IAASuperpixels, CenterCrop, ChannelDropout, ChannelShuffle,
                           CLAHE, CoarseDropout, Downscale, ElasticTransform, Equalize,
                           FancyPCA, Flip, GaussianBlur, GaussNoise, GlassBlur,
                           GridDistortion, GridDropout, Posterize, RandomBrightness,
                           RandomContrast, RandomFog, RandomGamma, RandomGridShuffle, RandomRain, RandomRotate90,
                           RandomShadow, RandomSnow, RandomSunFlare, Transpose)

In [ ]:
#image augmentations --> will try CLAHE p=1 first
#IAAAdditiveGaussianNoise, IAAAffine, IAASuperpixels, CLAHE, CoarseDropout Elastic Transform, Equalize, FancyPCA, Flip, GaussianBlur
#GaussNoise, GlassBlur, GridDistortion, RandomGamma RandomRotate90

AUGMENTATIONS = Compose([IAAAdditiveGaussianNoise(loc=0, p=0.2),
                        IAAAffine(p=0.2),
                        IAASuperpixels(p=0.2),
                        CLAHE(p=1),
                        #CoarseDropout(max_holes=5, max_height=24, max_width=40, min_holes = 2, min_height = 10, min_width = 30, fill_value=0), no p so remove
                        ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, approximate=False, p=0.2),
                        #Equalize(mode='cv', by_channels=True, mask=None, mask_params=()),
                        FancyPCA(alpha=0.1, p=0.2),
                        Flip(p=0.2),
                        GaussianBlur(blur_limit=7, p=0.2),
                        GaussNoise(var_limit=(10.0, 50.0), mean=0, p=0.2),
                        #GlassBlur(sigma=0.7, max_delta=4, iterations=2, mode='fast'), took out for now cuz no p=0.2
                        GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, p=0.2),
                        RandomGamma(gamma_limit=(80,120), eps=None, p=0.1),
                        RandomRotate90(p=0.3),
                        ])

In [ ]:
#image augments in imagedatagenerator --> https://machinelearningmastery.com/image-augmentation-deep-learning-keras/
#https://www.curiousily.com/posts/image-data-augmentation-for-tensorflow-2-keras-and-pytorch-with-albumentations-in-python/

In [ ]:
pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *

In [ ]:
Train_Generator = ImageDataAugmentor(rescale=1./255, 
                             #horizontal_flip = True, 
                             #rotation_range = 40, 
                             #shear_range = 0.1, 
                             #width_shift_range=0.2, 
                             #height_shift_range=0.2,
                             augment = AUGMENTATIONS,
                             preprocess_input=None)

Val_Generator = ImageDataGenerator(rescale=1./255)

Train_Data = Train_Generator.flow_from_dataframe(dataframe = train_gen, 
                                              x_col = 'image_dir', 
                                              y_col = 'target', 
                                              class_mode = 'raw', 
                                              target_size = (224, 224),
                                              color_mode = 'rgb', 
                                              batch_size = 8, 
                                              seed = 316, 
                                              shuffle = True)

Val_Data = Val_Generator.flow_from_dataframe(dataframe = val_gen, 
                                              x_col = 'image_dir', 
                                              y_col = 'target', 
                                              class_mode = 'raw', 
                                              target_size = (224, 224),
                                              color_mode = 'rgb', 
                                              batch_size = 8, 
                                              seed = 316, 
                                              shuffle = True)

In [ ]:
#from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.metrics import AUC
from keras.activations import sigmoid
from keras.optimizers import SGD, Adam, Adamax
from keras import Model

In [ ]:
!pip install -q efficientnet

In [ ]:
import efficientnet.tfkeras as efn 
#library somebody built on efficientnet implementations on keras --> https://github.com/qubvel/efficientnet

In [ ]:
#import vgg16 and add last layer to it
enb6 = efn.EfficientNetB6(weights='noisy-student', include_top=False, input_shape=(256, 256, 3), pooling = 'avg')
x = tf.keras.layers.Flatten()(enb6.output)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(enb6.input, output)

In [ ]:
https://www.google.com/search?q=sw+%EA%B0%9C%EB%B0%9C%EB%B3%91&oq=sw+%EA%B0%9C%EB%B0%9C%EB%B3%91&aqs=chrome..69i57j69i61.6026j0j7&client=ubuntu&sourceid=chrome&ie=UTF-8#future improvements?? --> https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [ ]:
import keras.backend as K

In [ ]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
opt = tf.keras.optimizers.Adam(lr = 1e-5)
model.compile(loss=focal_loss(), metrics=[tf.keras.metrics.AUC()],optimizer=opt)

In [ ]:
nb_epochs = 100
batch_size=8
nb_train_steps = train_gen.shape[0]//batch_size
nb_val_steps=val_gen.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
!pip install livelossplot
from livelossplot.inputs.tf_keras import PlotLossesCallback

In [ ]:
csv_logger = tf.keras.callbacks.CSVLogger('training.log', separator=',', append=False)

In [ ]:
#callback for early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10, verbose = 1, restore_best_weights = True)

In [ ]:
#bottlneck in imagedatagenerator
#https://medium.com/@joelognn/improving-cnn-training-times-in-keras-7405baa50e09
#might want to use tpus instead like the dude that finished calc in 2 minutes

In [ ]:
#fit_generator is better for larger, more complex data --> https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/
#fit_generator vs fit --> https://medium.com/difference-engine-ai/keras-a-thing-you-should-know-about-keras-if-you-plan-to-train-a-deep-learning-model-on-a-large-fdd63ce66bd2
cb=[PlotLossesCallback(), csv_logger, early_stopping] #only csv_logger since we already instantiated CSVLogger beforehand
model.fit_generator(
    Train_Data,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=Val_Data,
    callbacks=cb,
    validation_steps=nb_val_steps) #workers is number of cpu cores, which is 2 here for a gpu kernel, 4 for a only cpu kernel #gpu def helps lmaooo

In [ ]:
#saving model thru tf
model_filename = 'saved_model.tf'
model.save(model_filename)

In [ ]:
log_data = pd.read_csv('training.log', sep=',', engine='python')

In [ ]:
from PIL import Image

In [ ]:
#submission column

test_list = test['image_name'].tolist()
test_path = '../input/siim-isic-melanoma-classification/jpeg/test/'
final_target = []
test_img = []
size = 256, 256


'''
for name in image_list:
    pholder = Image.open(base_path + name + '.jpg')
    pholder = pholder.resize(size) #resizes image w/ pillow
    pholder = np.asarray(pholder)
    train_img.append(pholder)
'''
for name in test_list:
    pholder1 = Image.open(test_path + name + '.jpg')
    pholder1 = pholder1.resize(size) #resizes image w/ pillow
    pholder1 = np.asarray(pholder1)
    pholder1 = pholder1.astype('float32') 
    pholder1 /= 255.0 #normalize
    pholder1 = [pholder1]
    pholder1 = np.asarray(pholder1)
    pholder1 = model.predict(pholder1)
    pholder1 = pholder1[0][0]
    final_target.append(pholder1)
    
    
    
    
test_list = np.asarray(test_list)
final_target = np.asarray(final_target)
submission = pd.DataFrame({'image_name':test_list, 'target':final_target})

    #pholder = [pholder]
    #pholder = np.asarray(pholder)
    #pholder = mel_model.predict(pholder)
    #pholder = pholder[0][0] #to get value out of double bracket array
    
'''test_img = np.asarray(test_img)    
prediction = mel_model.predict(test_img)    

prediction = np.asarray(prediction)
final_target = np.asarray(final_target)

#submission = pd.DataFrame({'image_name':test_list, 'target':final_target})'''

In [ ]:
submission.to_csv('submission.csv', index=False)